# openbb_akshare

Version: 0.4.12

In [2]:
import pandas as pd

def get_df(obj) -> pd.DataFrame:
    d = [item.model_dump() for item in obj.results]
    return pd.DataFrame(d)

## Caching `openbb_akshare` data

There are query limitations on the number of requests that can be made to the akshare API. We need to cache the data to avoid hitting these limits.

References:
- https://github.com/ranaroussi/yfinance/blob/main/yfinance/cache.py
- https://github.com/ValueRaider/yfinance-cache

In [19]:
from openbb_core.app.constants import USER_SETTINGS_PATH
from openbb_core.app.model.defaults import Defaults
from openbb_core.app.model.preferences import Preferences
from openbb_core.app.model.profile import Profile

from openbb import obb
# Data can be cached in the OpenBB user preferences directory.
# obb.user.preferences.data_directory
# obb.user.preferences.cache_directory
# obb.user.preferences.export_directory

In [1]:
from openbb_core.app.utils import get_user_cache_directory
get_user_cache_directory()

'C:\\Users\\Roger\\OpenBBUserData\\cache'

## Implementation

#### Currency
##### CurrencyHistorical

AKShare API: https://akshare.akfamily.xyz/data/fx/fx.html#id2

东财数据 ： https://quote.eastmoney.com/cnyrate/EURCNYC.html

In [3]:
from openbb import obb
h_obj = obb.currency.price.historical(symbol='EURUSD', provider='akshare')

In [4]:
currency_data = get_df(h_obj)
currency_data.head()

,date,open,high,low,close,volume,vwap,change,symbol,name
0,1999-01-01,1.174,1.180,1.166,1.180,None,None,0.00,EURUSD,欧元兑美元
1,1999-01-04,1.175,1.190,1.174,1.179,None,None,1.36,EURUSD,欧元兑美元
2,1999-01-05,1.179,1.184,1.177,1.182,None,None,0.59,EURUSD,欧元兑美元
3,1999-01-06,1.182,1.182,1.172,1.175,None,None,0.85,EURUSD,欧元兑美元
4,1999-01-07,1.175,1.177,1.156,1.166,None,None,1.79,EURUSD,欧元兑美元


In [9]:
import akshare as ak

forex_hist_em_df = ak.forex_hist_em(symbol="USDCNH")
#forex_hist_em_df.to_excel("docs/forex_hist_em.xlsx")

In [10]:
#forex_hist_em_df[["日期", "今开", "最高", "最低"]].head()
forex_hist_em_df.head()

,日期,代码,名称,今开,最新价,最高,最低,振幅
0,2010-08-23,USDCNH,美元兑离岸人民币,6.7200,6.7250,6.7250,6.7200,0.00
1,2010-08-24,USDCNH,美元兑离岸人民币,6.7230,6.7265,6.7265,6.7230,0.05
2,2010-08-25,USDCNH,美元兑离岸人民币,6.7325,6.7325,6.7325,6.7325,0.00
3,2010-08-26,USDCNH,美元兑离岸人民币,6.7500,6.7450,6.7500,6.7450,0.07
4,2010-08-27,USDCNH,美元兑离岸人民币,6.7530,6.7530,6.7530,6.7530,0.00


##### CurrencySnapshots

AKShare API: https://akshare.akfamily.xyz/data/fx/fx.html#id1

东财数据: https://quote.eastmoney.com/center/gridlist.html#forex_all


In [18]:
from openbb import obb
s_obj = obb.currency.snapshots(provider='akshare')

In [19]:
snapshot_data = get_df(s_obj)
snapshot_data.head()

,base_currency,counter_currency,last_rate,open,high,low,close,volume,prev_close,change,change_percent
0,USDARS,美元兑阿根廷比索,1231.4321,1241.6196,1244.2923,1231.0077,None,None,1231.5053,10.1143,0.0082
1,CNYKRWC,人民币韩元中间价,190.6100,190.6100,190.6100,190.6100,None,None,189.3600,1.2500,0.0066
2,JPYZAR,100日元兑南非兰特,12.0828,12.1649,12.2157,12.0753,None,None,12.0901,0.0748,0.0062
3,CHFZAR,瑞郎兑南非兰特,22.0152,22.1649,22.2293,21.9719,None,None,22.0297,0.1352,0.0061
4,EURZAR,欧元兑南非兰特,20.5805,20.7203,20.7747,20.5799,None,None,20.5999,0.1204,0.0058


In [20]:
import akshare as ak

forex_spot_em_df = ak.forex_spot_em()
#forex_spot_em_df.to_excel("docs/forex_spot_em.xlsx")

In [21]:
new_df=forex_spot_em_df.drop(columns=["序号"])
new_df.head()

,代码,名称,最新价,涨跌额,涨跌幅,今开,最高,最低,昨收
0,USDARS,美元兑阿根廷比索,1231.4321,10.1143,0.82,1241.6196,1244.2923,1231.0077,1231.5053
1,CNYKRWC,人民币韩元中间价,190.6100,1.2500,0.66,190.6100,190.6100,190.6100,189.3600
2,JPYZAR,100日元兑南非兰特,12.0828,0.0748,0.62,12.1649,12.2157,12.0753,12.0901
3,CHFZAR,瑞郎兑南非兰特,22.0152,0.1352,0.61,22.1649,22.2293,21.9719,22.0297
4,EURZAR,欧元兑南非兰特,20.5805,0.1204,0.58,20.7203,20.7747,20.5799,20.5999


#### CashFlowStatement

##### 现金流量表-东财 按年度

AKShare API: https://akshare.akfamily.xyz/data/stock/stock.html#id188

东财数据: https://emweb.securities.eastmoney.com/PC_HSF10/NewFinanceAnalysis/Index?type=web&code=sh600519#lrb-0

In [22]:
from openbb import obb
cash_obj = obb.equity.fundamental.cash(symbol="600028", provider="akshare")
cash_flow = get_df(cash_obj)
cash_flow.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,period_ending,fiscal_period,fiscal_year,reported_currency,TOTAL_OPERATE_INFLOW,TOTAL_OPERATE_OUTFLOW,NETCASH_OPERATE,TOTAL_INVEST_INFLOW,TOTAL_INVEST_OUTFLOW,NETCASH_INVEST,TOTAL_FINANCE_INFLOW,TOTAL_FINANCE_OUTFLOW,NETCASH_FINANCE,NETPROFIT
0,2024-12-31,年报,None,CNY,3639081000000,3489721000000,149360000000,104645000000,265885000000,-161240000000,689198000000,708435000000,-19237000000,57547000000
1,2023-12-31,年报,None,CNY,3754346000000,3592871000000,161475000000,113746000000,269611000000,-155865000000,701339000000,678607000000,22732000000,70046000000
2,2022-12-31,年报,None,CNY,3832043000000,3715774000000,116269000000,129359000000,224369000000,-95010000000,569352000000,609051000000,-39699000000,76614000000
3,2021-12-31,年报,None,CNY,3143608000000,2918434000000,225174000000,64837000000,210035000000,-145198000000,357593000000,415535000000,-57942000000,85030000000
4,2020-12-31,年报,None,CNY,2511568000000,2343048000000,168520000000,134355000000,237005000000,-102650000000,563599000000,601109000000,-37510000000,42097000000


In [1]:
import akshare as ak

stock_cash_flow_sheet_by_yearly_em_df = ak.stock_cash_flow_sheet_by_yearly_em(symbol="SH600519")
#stock_cash_flow_sheet_by_yearly_em_df.T.to_excel("docs/cash_flow_em.xlsx")

  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
cash_flow_em = stock_cash_flow_sheet_by_yearly_em_df[["REPORT_DATE", "REPORT_TYPE", "CURRENCY",
    "TOTAL_OPERATE_INFLOW", "TOTAL_OPERATE_OUTFLOW", "NETCASH_OPERATE", 
    "TOTAL_INVEST_INFLOW", "TOTAL_INVEST_OUTFLOW", "NETCASH_INVEST", 
    "TOTAL_FINANCE_INFLOW", "TOTAL_FINANCE_OUTFLOW","NETCASH_FINANCE", "NETPROFIT"]]
cash_flow_em.head()

,REPORT_DATE,REPORT_TYPE,CURRENCY,TOTAL_OPERATE_INFLOW,TOTAL_OPERATE_OUTFLOW,NETCASH_OPERATE,TOTAL_INVEST_INFLOW,TOTAL_INVEST_OUTFLOW,NETCASH_INVEST,TOTAL_FINANCE_INFLOW,TOTAL_FINANCE_OUTFLOW,NETCASH_FINANCE,NETPROFIT
0,2024-12-31 00:00:00,年报,CNY,2.000863e+11,1.076226e+11,9.246369e+10,8.648630e+09,1.043383e+10,-1.785203e+09,NaN,7.106751e+10,-7.106751e+10,8.933473e+10
1,2023-12-31 00:00:00,年报,CNY,1.682562e+11,1.016629e+11,6.659325e+10,7.720217e+09,1.744463e+10,-9.724414e+09,NaN,5.888910e+10,-5.888910e+10,7.752148e+10
2,2022-12-31 00:00:00,年报,CNY,1.378159e+11,1.011173e+11,3.669860e+10,1.120691e+07,5.548033e+09,-5.536826e+09,NaN,5.742453e+10,-5.742453e+10,6.537604e+10
3,2021-12-31 00:00:00,年报,CNY,1.316210e+11,6.759231e+10,6.402868e+10,1.938686e+07,5.581833e+09,-5.562446e+09,NaN,2.656414e+10,-2.656414e+10,5.572053e+10
4,2020-12-31 00:00:00,年报,CNY,1.135109e+11,6.184178e+10,5.166907e+10,3.220777e+08,2.127305e+09,-1.805227e+09,NaN,2.412754e+10,-2.412754e+10,4.952333e+10


#### IncomeStatement

In [23]:
from openbb import obb
income_obj = obb.equity.fundamental.income(symbol="600028", provider="akshare")
income_df = get_df(income_obj)
income_df.head()

  0%|          | 0/6 [00:00<?, ?it/s]

,period_ending,fiscal_period,fiscal_year,reported_currency,cost_of_revenue,total_operating_income,total_pre_tax_income,income_tax_expense,consolidated_net_income,basic_earnings_per_share,diluted_earnings_per_share
0,2024-12-31,年报,None,CNY,3.021457e+12,3.074562e+12,7.051300e+10,1.296600e+10,5.754700e+10,0.415,0.415
1,2023-12-31,年报,None,CNY,3.138351e+12,3.212215e+12,8.611600e+10,1.607000e+10,7.004600e+10,0.505,0.505
2,2022-12-31,年报,None,CNY,3.232467e+12,3.318168e+12,9.451500e+10,1.790100e+10,7.661400e+10,0.555,0.555
3,2021-12-31,年报,None,CNY,2.628882e+12,2.740884e+12,1.083480e+11,2.331800e+10,8.503000e+10,0.588,0.588
4,2020-12-31,年报,None,CNY,2.081582e+12,2.104724e+12,4.844100e+10,6.344000e+09,4.209700e+10,0.275,0.275


##### 利润表-东财

AKShare API:
https://akshare.akfamily.xyz/data/stock/stock.html#id185

东财数据:
https://emweb.securities.eastmoney.com/PC_HSF10/NewFinanceAnalysis/Index?type=web&code=sh600519#lrb-0

In [24]:
import akshare as ak

stock_profit_sheet_by_yearly_em_df = ak.stock_profit_sheet_by_yearly_em(symbol="SH600519")
# stock_profit_sheet_by_yearly_em_df.T.to_excel("docs/income_statement_em.xlsx")

  0%|          | 0/6 [00:00<?, ?it/s]

In [25]:

income_statement_em = stock_profit_sheet_by_yearly_em_df[["REPORT_DATE", "REPORT_TYPE", "CURRENCY", "TOTAL_OPERATE_COST", "TOTAL_OPERATE_INCOME", "TOTAL_PROFIT", "INCOME_TAX", "NETPROFIT", "BASIC_EPS", "DILUTED_EPS"]]
income_statement_em.head()

,REPORT_DATE,REPORT_TYPE,CURRENCY,TOTAL_OPERATE_COST,TOTAL_OPERATE_INCOME,TOTAL_PROFIT,INCOME_TAX,NETPROFIT,BASIC_EPS,DILUTED_EPS
0,2024-12-31 00:00:00,年报,CNY,5.452397e+10,1.741441e+11,1.196386e+11,3.030385e+10,8.933473e+10,68.64,68.64
1,2023-12-31 00:00:00,年报,CNY,4.696089e+10,1.505603e+11,1.036626e+11,2.614108e+10,7.752148e+10,59.49,59.49
2,2022-12-31 00:00:00,年报,CNY,3.974831e+10,1.275540e+11,8.770149e+10,2.232545e+10,6.537604e+10,49.93,49.93
3,2021-12-31 00:00:00,年报,CNY,3.477690e+10,1.094643e+11,7.452803e+10,1.880750e+10,5.572053e+10,41.76,41.76
4,2020-12-31 00:00:00,年报,CNY,3.130513e+10,9.799324e+10,6.619694e+10,1.667361e+10,4.952333e+10,37.17,37.17


#### BalanceSheet

In [26]:
from openbb import obb
balance_obj = obb.equity.fundamental.balance(symbol="600028", provider="akshare")
balance_df = get_df(balance_obj)
balance_df.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,period_ending,fiscal_period,fiscal_year,totalAssets,totalDebt,totalEquity
0,2024-12-31,年报,2024,2084771000000,1108478000000,976293000000
1,2023-12-31,年报,2023,2026674000000,1068019000000,958655000000
2,2022-12-31,年报,2022,1951121000000,1010664000000,940457000000
3,2021-12-31,年报,2021,1889255000000,973214000000,916041000000
4,2020-12-31,年报,2020,1738896000000,850176000000,888720000000


##### 财务报表-新浪

API:  https://akshare.akfamily.xyz/data/stock/stock.html#id180

Data: https://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinanceSummary/stockid/600600/displaytype/4.phtml?source=fzb&qq-pf-to=pcqq.group


In [ ]:
import akshare as ak

stock_financial_report_sina_df = ak.stock_financial_report_sina(stock="sh600600", symbol="资产负债表")
balance_sheet_sina = stock_financial_report_sina_df.T
#balance_sheet_sina.to_excel("docs/balance_sheet_sina_df.xlsx")

In [28]:
# 报告日, 资产总计(totalAssets), 负债合计(totalDebt), 所有者权益(或股东权益)合计 (totalEquity)
import pandas as pd
balance_sheet_df = stock_financial_report_sina_df[["报告日", "资产总计", "负债合计", "所有者权益(或股东权益)合计"]]
balance_sheet_df["报告日"] = pd.to_datetime(balance_sheet_df["报告日"], format="%Y%m%d", errors="coerce")
balance_sheet_df.head()

C:\Users\roger\AppData\Local\Temp\ipykernel_27876\3636249029.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df["报告日"] = pd.to_datetime(balance_sheet_df["报告日"], format="%Y%m%d", errors="coerce")


,报告日,资产总计,负债合计,所有者权益(或股东权益)合计
0,2025-03-31,51291152685.0,19733489350.0,31557663335.0
1,2024-12-31,51420385832.0,21560843821.0,29859542011.0
2,2024-09-30,50371239687.0,19698994899.0,30672244788.0
3,2024-06-30,52805909965.0,23590301039.0,29215608926.0
4,2024-03-31,50459434299.0,20614109997.0,29845324302.0


##### 财务报表-东财
https://emweb.securities.eastmoney.com/PC_HSF10/NewFinanceAnalysis/Index?type=web&code=sh600519#lrb-0

###### A股

In [ ]:
import akshare as ak

stock_balance_sheet_by_yearly_em_df = ak.stock_balance_sheet_by_yearly_em(symbol="SH600519")
balance_sheet_em = stock_balance_sheet_by_yearly_em_df.T
#balance_sheet_em.to_excel("docs/balance_sheet_em.xlsx")

  0%|          | 0/6 [00:00<?, ?it/s]

In [2]:
balance_sheet_em = stock_balance_sheet_by_yearly_em_df[["REPORT_DATE", "REPORT_TYPE", "REPORT_DATE_NAME", "TOTAL_ASSETS", "TOTAL_LIABILITIES", "TOTAL_EQUITY"]]
balance_sheet_em['REPORT_DATE_NAME'] = balance_sheet_em['REPORT_DATE_NAME'].str.extract('(\d{4})')[0].astype(int)
balance_sheet_em.head()

C:\Users\Roger\AppData\Local\Temp\ipykernel_31836\1028356269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_em['REPORT_DATE_NAME'] = balance_sheet_em['REPORT_DATE_NAME'].str.extract('(\d{4})')[0].astype(int)


,REPORT_DATE,REPORT_TYPE,REPORT_DATE_NAME,TOTAL_ASSETS,TOTAL_LIABILITIES,TOTAL_EQUITY
0,2024-12-31 00:00:00,年报,2024,2.989446e+11,5.693326e+10,2.420113e+11
1,2023-12-31 00:00:00,年报,2023,2.726997e+11,4.904319e+10,2.236565e+11
2,2022-12-31 00:00:00,年报,2022,2.545008e+11,4.956274e+10,2.049381e+11
3,2021-12-31 00:00:00,年报,2021,2.551682e+11,5.821069e+10,1.969575e+11
4,2020-12-31 00:00:00,年报,2020,2.133958e+11,4.567513e+10,1.677207e+11


###### 港股

In [ ]:
import akshare as ak

stock_financial_hk_report_em_df = ak.stock_financial_hk_report_em(stock="00700", symbol="资产负债表", indicator="年度")
#stock_financial_hk_report_em_df.T.to_excel("docs/balance_sheet_em_hk.xlsx")

In [20]:
balance_sheet_em_hk = stock_financial_hk_report_em_df[["STD_REPORT_DATE", "FISCAL_YEAR", "AMOUNT"]]
balance_sheet_em_hk.head()

,STD_REPORT_DATE,FISCAL_YEAR,AMOUNT
0,2024-12-31 00:00:00,12-31,8.018500e+10
1,2024-12-31 00:00:00,12-31,8.010000e+08
2,2024-12-31 00:00:00,12-31,1.961270e+11
3,2024-12-31 00:00:00,12-31,2.311700e+10
4,2024-12-31 00:00:00,12-31,1.230200e+10


##### 财务报表-同花顺
https://akshare.akfamily.xyz/data/stock/stock.html#id191
https://basic.10jqka.com.cn/000063/finance.html

In [ ]:
# 财务报表-同花顺 资产负债表

import akshare as ak

stock_financial_debt_ths_df = ak.stock_financial_debt_ths(symbol="000063", indicator="按年度")
balance_sheet_ths=stock_financial_debt_ths_df.T

In [23]:
balance_sheet_ths.to_excel("balance_sheet_ths.xlsx")

#### AvailableIndices

In [2]:
import akshare as ak

index_stock_info_df = ak.index_stock_info()


In [5]:
index_stock_info_df["currency"] = "CNY"
index_stock_info_df

,index_code,display_name,publish_date,currency
0,000001,上证指数,1991-07-15,CNY
1,000002,A股指数,1992-02-21,CNY
2,000003,B股指数,1992-02-21,CNY
3,000004,工业指数,1993-05-03,CNY
4,000005,商业指数,1993-05-03,CNY
...,...,...,...,...
720,399994,中证信息安全主题指数,2015-03-12,CNY
721,399995,中证基建工程指数,2015-03-12,CNY
722,399996,中证智能家居指数,2014-09-17,CNY
723,399997,中证白酒指数,2015-01-21,CNY


In [1]:
from openbb import obb
indx = obb.index.available(provider="akshare")
indx.results[0]

AKShareAvailableIndicesData(name=上证指数, currency=CNY, publish_date=1991-07-15, symbol=000001)

In [4]:
from openbb import obb
indx = obb.index.available(provider="yfinance")
indx.results[0]

YFinanceAvailableIndicesData(name=S&P 500 Index, currency=None, code=sp500, symbol=^GSPC)

In [2]:
fmp_indx  = obb.index.available(provider="fmp")

In [3]:
fmp_indx.results[0]

FMPAvailableIndicesData(name=S&P/TSX Capped Industrials Index, currency=CAD, stock_exchange=Toronto Stock Exchange, exchange_short_name=INDEX, symbol=^TTIN)

#### historical_dividends



##### Helpers
- `get_a_dividends`
- `get_hk_dividends`

In [3]:
import logging
from openbb_akshare.utils.tools import setup_logger

setup_logger()
logger = logging.getLogger(__name__)


In [4]:
from openbb_akshare.utils.helpers import get_a_dividends, get_hk_dividends
div = get_a_dividends(symbol="000002")

c:\src\tmp\akshare\openbb_akshare\utils\helpers.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker['amount'] = div_df['分红方案说明'].apply(
c:\src\tmp\akshare\openbb_akshare\utils\helpers.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker.rename(columns={'实施公告日': "report_date",


In [10]:
from openbb_akshare.utils.helpers import get_hk_dividends
div_hk = get_hk_dividends(symbol="02880")

f:\src\tmp\akshare\openbb_akshare\utils\helpers.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker['amount'] = div_df['方案'].apply(
f:\src\tmp\akshare\openbb_akshare\utils\helpers.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker.rename(columns={'公告日期': "report_date",


##### 分红情况-同花顺[](https://akshare.akfamily.xyz/data/stock/stock.html#id157 "Link to this heading")

接口: stock_fhps_detail_ths

目标地址: https://basic.10jqka.com.cn/new/603444/bonus.html

描述: 同花顺-分红情况

In [13]:
import akshare as ak

stock_fhps_detail_ths_df = ak.stock_fhps_detail_ths(symbol="600325")


In [14]:
stock_fhps_detail_ths_df.dropna(inplace=True)
stock_fhps_detail_ths_df.head()

,报告期,董事会日期,股东大会预案公告日期,实施公告日,分红方案说明,A股股权登记日,A股除权除息日,分红总额,方案进度,股利支付率,税前分红率
1,2004年报,2005-01-22,2005-02-24,2005-04-12,10派1元(含税),2005-04-15,2005-04-18,2000.00万,实施方案,22.22%,1.43%
2,2005中报,2005-08-30,2005-10-10,2005-10-22,10转增3股,2005-10-27,2005-10-28,--,实施方案,--,--
3,2005年报,2006-02-18,2006-03-21,2006-04-01,10派1元(含税),2006-04-06,2006-04-07,2600.00万,实施方案,16.39%,1.14%
7,2007年报,2008-01-23,2008-02-14,2008-02-21,10转10股派1元(含税),2008-02-26,2008-02-27,3159.58万,实施方案,8.2%,0.23%
9,2008年报,2009-01-23,2009-02-13,2009-03-26,10派1.1元(含税),2009-04-01,2009-04-02,8987.50万,实施方案,12.5%,0.66%


##### 分红配送详情-港股-同花顺[](https://akshare.akfamily.xyz/data/stock/stock.html#id158 "Link to this heading")

接口: stock_hk_fhpx_detail_ths

目标地址: https://stockpage.10jqka.com.cn/HK0700/bonus/

描述: 同花顺-港股-分红派息

In [15]:
import akshare as ak

stock_hk_fhpx_detail_ths_df = ak.stock_hk_fhpx_detail_ths(symbol="2880")


In [16]:
stock_hk_fhpx_detail_ths_df.dropna(inplace=True)
stock_hk_fhpx_detail_ths_df.head()

,公告日期,方案,除净日,派息日,过户日期起止日-起始,过户日期起止日-截止,类型,进度,以股代息
1,2007-06-13,每股0.06122港元,2007-05-10,2007-06-29,2007-05-12,2007-05-17,年报,实施完成,否
3,2008-06-18,每股0.09051港元,2008-05-15,2008-07-16,2008-05-19,2008-05-23,年报,实施完成,否
5,2009-06-19,每股0.10207港元,2009-05-18,2009-07-31,2009-05-20,2009-05-25,年报,实施完成,否
7,2010-06-18,每股人民币0.25元,2010-05-17,2010-07-31,2010-05-19,2010-06-18,年报,实施完成,否
9,2011-07-20,每股0.060221港元,2011-05-23,2011-08-19,2011-05-25,2011-06-24,年报,实施完成,否


##### obb.equity.fundamental.dividends

In [17]:
from openbb import obb
from openbb_akshare.utils.tools import setup_logger
setup_logger()


In [18]:
dividends = obb.equity.fundamental.dividends(symbol='600325', provider='akshare')
dividends.results[0]

2025-07-03 16:44:18,582 - openbb_akshare.models.historical_dividends - INFO - Fetched historical dividends:
[{'report_date': datetime.date(2005, 4, 12), 'description': '10派1元(含税)', 'record_date': datetime.date(2005, 4, 15), 'ex_dividend_date': datetime.date(2005, 4, 18), 'amount': 0.1}, {'report_date': datetime.date(2005, 10, 22), 'description': '10转增3股', 'record_date': datetime.date(2005, 10, 27), 'ex_dividend_date': datetime.date(2005, 10, 28), 'amount': 0.0}, {'report_date': datetime.date(2006, 4, 1), 'description': '10派1元(含税)', 'record_date': datetime.date(2006, 4, 6), 'ex_dividend_date': datetime.date(2006, 4, 7), 'amount': 0.1}, {'report_date': datetime.date(2008, 2, 21), 'description': '10转10股派1元(含税)', 'record_date': datetime.date(2008, 2, 26), 'ex_dividend_date': datetime.date(2008, 2, 27), 'amount': 0.1}, {'report_date': datetime.date(2009, 3, 26), 'description': '10派1.1元(含税)', 'record_date': datetime.date(2009, 4, 1), 'ex_dividend_date': datetime.date(2009, 4, 2), 'amount': 0

AKShareHistoricalDividendsData(ex_dividend_date=2005-04-18, amount=0.1, reported_date=None, description=10派1元(含税), record_date=2005-04-15, declaration_date=None, report_date=2005-04-12)

In [1]:
from openbb import obb
dividends = obb.equity.fundamental.dividends(symbol='AAPL', provider='fmp')
dividends.results[0]

FMPHistoricalDividendsData(ex_dividend_date=2025-05-12, amount=0.26, label=May 12, 25, adj_dividend=0.26, record_date=2025-05-12, payment_date=2025-05-15, declaration_date=2025-05-01)

In [ ]:
yf_dividends = obb.equity.fundamental.dividends(symbol='AAPL', provider='yfinance')
yf_dividends.results[0]

YFinanceHistoricalDividendsData(ex_dividend_date=1987-05-11, amount=0.000536)

In [ ]:
import akshare as ak

stock_fhps_detail_em_df = ak.stock_fhps_detail_em(symbol="600325")
stock_fhps_detail_em_df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

,报告期,业绩披露日期,送转股份-送转总比例,送转股份-送股比例,送转股份-转股比例,现金分红-现金分红比例,现金分红-现金分红比例描述,现金分红-股息率,每股收益,每股净资产,每股公积金,每股未分配利润,净利润同比增长,总股本,预案公告日,股权登记日,除权除息日,方案进度,最新公告日期
0,2004-12-31,2005-01-22,0.0,0.0,0.0,1.0,"10派1.00元(含税,扣税后0.80元)",0.014327,0.45,4.948,2.069137,1.242755,52.515641,200000000,NaT,2005-04-15,2005-04-18,实施分配,2005-04-12
1,2005-06-30,2005-08-30,3.0,0.0,3.0,NaN,10转3.00,NaN,0.35,5.202,2.069137,1.596814,178.905414,200000000,NaT,2005-10-27,2005-10-28,实施分配,2005-10-22
2,2005-12-31,2006-02-18,0.0,0.0,0.0,1.0,"10派1.00元(含税,扣税后0.90元)",0.011364,0.61,4.220,1.360874,1.267908,50.144752,260000000,NaT,2006-04-06,2006-04-07,实施分配,2006-04-01
3,2007-12-31,2008-01-23,10.0,0.0,10.0,1.0,"10转10.00派1.00元(含税,扣税后0.90元)",0.002350,1.22,6.230,2.266939,2.444449,145.474112,315957539,2008-01-23,2008-02-26,2008-02-27,实施分配,2008-02-21
4,2008-12-31,2009-01-23,0.0,0.0,0.0,1.1,"10派1.10元(含税,扣税后0.99元)",0.006575,0.88,5.510,3.186507,1.073908,81.609495,817045620,2009-01-23,2009-04-01,2009-04-02,实施分配,2009-03-26


#### equity_historical

In [19]:
from openbb import obb
prices = obb.equity.price.historical(symbol='AAPL', start_date="2025-06-01", end_date="2025-06-10", provider="fmp")
prices.results

[FMPEquityHistoricalData(date=2025-06-02, open=200.28, high=202.13, low=200.12, close=201.7, volume=35423300, vwap=201.0575, adj_close=201.7, unadjusted_volume=35423300.0, change=1.42, change_percent=0.0070901),
 FMPEquityHistoricalData(date=2025-06-03, open=201.35, high=203.77, low=200.96, close=203.27, volume=46381600, vwap=202.3375, adj_close=203.27, unadjusted_volume=46381600.0, change=1.92, change_percent=0.0095356),
 FMPEquityHistoricalData(date=2025-06-04, open=202.91, high=206.24, low=202.1, close=202.82, volume=43604000, vwap=203.5175, adj_close=202.82, unadjusted_volume=43604000.0, change=-0.09, change_percent=-0.0004435464),
 FMPEquityHistoricalData(date=2025-06-05, open=203.5, high=204.75, low=200.15, close=200.63, volume=55221235, vwap=202.2575, adj_close=200.63, unadjusted_volume=55221235.0, change=-2.87, change_percent=-0.0141),
 FMPEquityHistoricalData(date=2025-06-06, open=203.0, high=205.7, low=202.05, close=203.92, volume=46607700, vwap=203.6675, adj_close=203.92, un

In [20]:
prices = obb.equity.price.historical(symbol='00001', start_date="2025-06-01", end_date="2025-06-10", provider="akshare")
prices.results

[AKShareEquityHistoricalData(date=2025-06-02, open=43.95, high=44.15, low=43.25, close=43.9, volume=5170410, vwap=None, change=-0.25, change_percent=-0.57, amount=226247856.0),
 AKShareEquityHistoricalData(date=2025-06-03, open=43.95, high=46.35, low=43.95, close=45.8, volume=18222350, vwap=None, change=1.9, change_percent=4.33, amount=825981456.0),
 AKShareEquityHistoricalData(date=2025-06-04, open=45.75, high=45.9, low=44.8, close=44.95, volume=7961576, vwap=None, change=-0.85, change_percent=-1.86, amount=359758448.0),
 AKShareEquityHistoricalData(date=2025-06-05, open=45.3, high=45.8, low=45.25, close=45.65, volume=5753585, vwap=None, change=0.7, change_percent=1.56, amount=262236571.0),
 AKShareEquityHistoricalData(date=2025-06-06, open=46.05, high=46.25, low=45.6, close=45.9, volume=7030219, vwap=None, change=0.25, change_percent=0.55, amount=322902592.0),
 AKShareEquityHistoricalData(date=2025-06-09, open=46.0, high=46.3, low=45.85, close=46.1, volume=6959874, vwap=None, change=

In [21]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="000001", period="daily", start_date="20250601", end_date='20250610', adjust="")
stock_zh_a_hist_df.head()

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2025-06-03,000001,11.54,11.81,11.91,11.53,2192480,2.580963e+09,3.29,2.16,0.25,1.13
1,2025-06-04,000001,11.82,11.84,11.88,11.78,1167960,1.383020e+09,0.85,0.25,0.03,0.60
2,2025-06-05,000001,11.88,11.67,11.91,11.66,1166803,1.369212e+09,2.11,-1.44,-0.17,0.60
3,2025-06-06,000001,11.70,11.70,11.79,11.68,682353,7.999000e+08,0.94,0.26,0.03,0.35
4,2025-06-09,000001,11.73,11.71,11.75,11.65,773255,9.050926e+08,0.85,0.09,0.01,0.40


In [4]:
stock_zh_a_hist_df.rename(columns={"日期": "date", "股票代码": "symbol", "开盘": "open", "收盘": "close", "最高": "high", "最低": "low", "成交量": "volume", "成交额": "amount", "涨跌幅":"change_percent", "涨跌额": "change"}, inplace=True)

In [5]:
stock_zh_a_hist_df

,date,symbol,open,close,high,low,volume,amount,振幅,change_percent,change,换手率
0,2025-06-03,000001,11.54,11.81,11.91,11.53,2192480,2.580963e+09,3.29,2.16,0.25,1.13
1,2025-06-04,000001,11.82,11.84,11.88,11.78,1167960,1.383020e+09,0.85,0.25,0.03,0.60
2,2025-06-05,000001,11.88,11.67,11.91,11.66,1166803,1.369212e+09,2.11,-1.44,-0.17,0.60
3,2025-06-06,000001,11.70,11.70,11.79,11.68,682353,7.999000e+08,0.94,0.26,0.03,0.35
4,2025-06-09,000001,11.73,11.71,11.75,11.65,773255,9.050926e+08,0.85,0.09,0.01,0.40
5,2025-06-10,000001,11.72,11.81,11.89,11.71,1428132,1.689019e+09,1.54,0.85,0.10,0.74


In [7]:
data1 = stock_zh_a_hist_df.drop(columns=["振幅"])
data1

,date,symbol,open,close,high,low,volume,amount,change_percent,change,换手率
0,2025-06-03,000001,11.54,11.81,11.91,11.53,2192480,2.580963e+09,2.16,0.25,1.13
1,2025-06-04,000001,11.82,11.84,11.88,11.78,1167960,1.383020e+09,0.25,0.03,0.60
2,2025-06-05,000001,11.88,11.67,11.91,11.66,1166803,1.369212e+09,-1.44,-0.17,0.60
3,2025-06-06,000001,11.70,11.70,11.79,11.68,682353,7.999000e+08,0.26,0.03,0.35
4,2025-06-09,000001,11.73,11.71,11.75,11.65,773255,9.050926e+08,0.09,0.01,0.40
5,2025-06-10,000001,11.72,11.81,11.89,11.71,1428132,1.689019e+09,0.85,0.10,0.74


In [1]:
from openbb_akshare.utils.helpers import ak_download
df = ak_download(symbol="000002", start_date="2025-06-01", end_date="2025-06-10", period="daily", adjust="")

In [2]:
df

,date,symbol,open,close,high,low,volume,amount,振幅,change_percent,change,换手率
0,2025-06-03,000002,6.58,6.58,6.60,6.54,646833,4.249952e+08,0.90,-0.90,-0.06,0.67
1,2025-06-04,000002,6.59,6.62,6.64,6.57,506252,3.343369e+08,1.06,0.61,0.04,0.52
2,2025-06-05,000002,6.61,6.64,6.66,6.59,511213,3.387872e+08,1.06,0.30,0.02,0.53
3,2025-06-06,000002,6.64,6.60,6.66,6.59,452611,2.993021e+08,1.05,-0.60,-0.04,0.47
4,2025-06-09,000002,6.60,6.62,6.64,6.58,443997,2.937162e+08,0.91,0.30,0.02,0.46
5,2025-06-10,000002,6.63,6.60,6.73,6.56,1041358,6.914206e+08,2.57,-0.30,-0.02,1.07


#### equity_quote

#### 实时行情数据-雪球[](https://akshare.akfamily.xyz/data/stock/stock.html#id21 "Link to this heading")

接口: stock_individual_spot_xq

目标地址: https://xueqiu.com/S/SH513520

描述: 雪球-行情中心-个股

限量: 单次获取指定 symbol 的最新行情数据

In [7]:
import akshare as ak

stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()


  0%|          | 0/57 [00:00<?, ?it/s]

In [ ]:
#stock_zh_a_spot_em_df.T.to_excel("../docs/stock_zh_a_spot_em.xlsx")
stock_quotes = stock_zh_a_spot_em_df[["代码", "名称", "最新价", "今开", "最高", "最低", "涨跌幅", "涨跌额", "成交量", "昨收"]]
stock_quotes.head()

,代码,名称,最新价,今开,最高,最低,涨跌幅,涨跌额,成交量,昨收
0,839680,*ST广道,5.87,4.20,5.87,4.18,29.87,1.35,196287.0,4.52
1,835305,*ST云创,16.33,12.91,17.15,12.91,22.78,3.03,242688.0,13.30
2,300436,广生堂,51.34,45.00,51.34,44.01,20.01,8.56,356788.0,42.78
3,688068,热景生物,194.81,169.81,194.81,169.81,20.00,32.47,52638.0,162.34
4,301389,隆扬电子,31.92,26.62,31.92,26.50,20.00,5.32,358897.0,26.60


In [1]:
from openbb import obb
data = obb.equity.price.quote(symbol='AAPL', provider='fmp')
data.results

[FMPEquityQuoteData(symbol=AAPL, asset_type=None, name=Apple Inc., exchange=NASDAQ, bid=None, bid_size=None, bid_exchange=None, ask=None, ask_size=None, ask_exchange=None, quote_conditions=None, quote_indicators=None, sales_conditions=None, sequence_number=None, market_center=None, participant_timestamp=None, trf_timestamp=None, sip_timestamp=None, last_price=213.55, last_tick=None, last_size=None, last_timestamp=2025-07-03 20:00:00+00:00, open=212.145, high=214.65, low=211.8101, close=None, volume=34955836, exchange_volume=None, prev_close=212.44, change=1.11, change_percent=0.005225, year_high=260.1, year_low=169.21, price_avg50=203.4426, price_avg200=223.1329, avg_volume=63501154, market_cap=3189540090000.0, shares_outstanding=14935800000, eps=7.09, pe=30.12, earnings_announcement=2025-07-30 10:59:00+00:00)]

In [3]:
from openbb_akshare.utils.tools import normalize_symbol
symbol = " 600325 "
symbol_b, symbol_f, market = normalize_symbol(symbol)
print(symbol_b, symbol_f, market)

600325 600325.SH SH


In [2]:
from openbb import obb

ak_data = obb.equity.price.quote("873416,000002,601288,00300", provider='akshare')
ak_data.to_dataframe().head()

,symbol,error,name,last_price,open,high,low,volume,prev_close,change,change_percent
0,873416,Symbol not found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000002,NaN,万 科Ａ,6.55,6.51,6.56,6.47,573798.0,6.52,0.03,0.46
2,601288,NaN,农业银行,6.10,6.08,6.11,6.04,1880481.0,6.08,0.02,0.33
3,00300,NaN,美的集团,76.45,77.00,77.00,75.80,2235641.0,76.50,-0.05,-0.07


In [3]:
# 北交所
#from openbb_akshare.utils.tools import setup_logger, normalize_symbol, get_symbol_base
#symbol, symbol_f, market = normalize_symbol('873416')
ak_data = obb.equity.price.quote("873416", provider='akshare')
ak_data.results

  0%|          | 0/57 [00:00<?, ?it/s]

Error fetching data for symbol 873416: 'dict' object has no attribute 'to_dict'


[]

In [4]:
# 上交所
ak_data = obb.equity.price.quote("600325", provider='akshare')
ak_data.results

  0%|          | 0/57 [00:00<?, ?it/s]

[AKShareEquityQuoteData(symbol=600325, asset_type=None, name=华发股份, exchange=None, bid=None, bid_size=None, bid_exchange=None, ask=None, ask_size=None, ask_exchange=None, quote_conditions=None, quote_indicators=None, sales_conditions=None, sequence_number=None, market_center=None, participant_timestamp=None, trf_timestamp=None, sip_timestamp=None, last_price=4.84, last_tick=None, last_size=None, last_timestamp=None, open=4.86, high=4.87, low=4.82, close=None, volume=228572.0, exchange_volume=None, prev_close=4.85, change=-0.01, change_percent=-0.21, year_high=None, year_low=None)]

In [5]:
# 深交所
ak_data = obb.equity.price.quote("000002", provider='akshare')
ak_data.results

  0%|          | 0/57 [00:00<?, ?it/s]

[AKShareEquityQuoteData(symbol=000002, asset_type=None, name=万  科Ａ, exchange=None, bid=None, bid_size=None, bid_exchange=None, ask=None, ask_size=None, ask_exchange=None, quote_conditions=None, quote_indicators=None, sales_conditions=None, sequence_number=None, market_center=None, participant_timestamp=None, trf_timestamp=None, sip_timestamp=None, last_price=6.44, last_tick=None, last_size=None, last_timestamp=None, open=6.48, high=6.52, low=6.41, close=None, volume=786688.0, exchange_volume=None, prev_close=6.46, change=-0.02, change_percent=-0.31, year_high=None, year_low=None)]

#### company_news
使用东财接口如下：
```python
import akshare as ak
stock_news_em_df = ak.stock_news_em(symbol="300059")
```

In [1]:
from openbb import obb

messages = obb.news.company("000002", provider="akshare")

In [4]:
for result in messages.results:
    print(f"Title: {result.title}")

Title: 万科A出售3010万股库存股 收款1.98亿元
Title: 万科A：6月10日出售2200万股A股库存股
Title: 万科A首次出售2200万A股库存股，筹资1.46亿元
Title: 万科A首次出售库存股回款1.46亿元
Title: 万科A首次出售！回款1.46亿元！
Title: 万科：首次出售2200万股A股库存股，所得资金约1.46亿元
Title: 万科A：深铁集团拟向公司提供不超过30亿元借款
Title: 万科A：深铁集团拟向公司提供不超30亿元借款
Title: 万科首次出售2200万股A股库存股，预计将持续多个交易日
Title: 万科A：截至5月31日尚未出售A股库存股
Title: 万科“回血”进行时 1.46亿元出售首批库存股
Title: 万科：深铁集团拟提供不超过30亿元股东借款
Title: 万科年内第五次向大股东借款 深铁出借近150亿托底
Title: 债市早参6月11日|年内土储专项债新发破千亿元；万科“回血”进行时，1.46亿元出售首批库存股
Title: 债市早参6月9日|本周万亿逆回购到期；万科再获深铁“输血”30亿
Title: 离开万科八年后 王石朋友圈“喊话”引关注
Title: 万科A：约定以不超60亿元万物云股票作为与深铁集团借款合同质押物
Title: 万科A：约定以不超60亿元万物云股票作为与深铁集团借款合同质押物
Title: 万科A：约定以不超60亿元万物云股票作为深铁集团借款合同质押物
Title: 万科A“割肉”
Title: 万科王石最新发声！
Title: 大股东驰援近150亿元，万科适度扩储
Title: 王石再发声！万科出售2200万股库存股“回血”
Title: 万科拟再度质押万物云股票 作为与深铁集团42亿元借款合同质押物
Title: 王石最新发声！万科晚间公告：“售股”回血！
Title: 万科A首次出售A股库存股 减持所得资金1.46亿元
Title: 回购三年后变现！万科首次出售A股库存股，1.46亿入账
Title: 万科首次出售2200万股A股库存股；南都物业独董高强被监察机关留置 | 房产早参
Title: 万科参股公司摘得武汉一宗宅地，成交价约4.69亿元
Title: 万科&长江产业集团以近5亿元摘得武汉1宗宅地
Title: 【中证十点半】万科首次出售

In [1]:
import akshare as ak

stock_news_em_df = ak.stock_news_em(symbol="300059")
print(stock_news_em_df)

       关键词                                               新闻标题  \
0   300059               证券ETF龙头(159993盘中涨超2%，机构：券商板块存在较大补涨空间   
1   300059    券商板块有望再迎“活水”，保险证券ETF515630强势涨超3%，机构：券商交易量或将持续改善   
2   300059  券商股持续活跃，证券公司ETF516730上涨2.39%，券商积极开展增持回购，机构继续看好...   
3   300059              中科曙光涨超3%，数字经济ETF560800盘中上涨，成交额超1000万元   
4   300059           计算机产业趋势利好积聚，数字经济ETF560800上涨0.53%，成交额超千万元   
..     ...                                                ...   
95  300059                                     8股特大单净流入资金超2亿元   
96  300059              资本市场格局有望重塑，金融科技ETF516860深V反弹，新国都涨超12%   
97  300059                    资金流向日报丨东方财富、比亚迪、中航成飞获融资资金买入排名前三   
98  300059              成交额超2000万元，金融科技ETF516860近1年涨幅排名可比基金首位   
99  300059        创业板50指数上涨0.29%，创业板50ETF华夏(159367)费率在可比基金中最低   

                                                 新闻内容                 发布时间  \
0   截至2025年6月11日 09:47，国证证券龙头指数(399437)成分股信达证券(601...  2025-06-11 10:21:24   
1   截至2025年6月11日 09:59，中证800证券保险指数(399966)强势上涨2.72...  2025-06-

In [2]:
stock_news_em_df

,关键词,新闻标题,新闻内容,发布时间,文章来源,新闻链接
0,300059,证券ETF龙头(159993盘中涨超2%，机构：券商板块存在较大补涨空间,截至2025年6月11日 09:47，国证证券龙头指数(399437)成分股信达证券(601...,2025-06-11 10:21:24,界面新闻,http://stock.eastmoney.com/a/20250611342755547...
1,300059,券商板块有望再迎“活水”，保险证券ETF515630强势涨超3%，机构：券商交易量或将持续改善,截至2025年6月11日 09:59，中证800证券保险指数(399966)强势上涨2.72...,2025-06-11 11:05:04,界面新闻,http://stock.eastmoney.com/a/20250611342759323...
2,300059,券商股持续活跃，证券公司ETF516730上涨2.39%，券商积极开展增持回购，机构继续看好...,截至2025年6月11日 14:14，中证证券公司30指数(931412)强势上涨2.36%...,2025-06-11 14:26:10,界面新闻,http://stock.eastmoney.com/a/20250611342767995...
3,300059,中科曙光涨超3%，数字经济ETF560800盘中上涨，成交额超1000万元,截至2025年6月11日 13:41，中证数字经济主题指数(931582)上涨0.36%，成...,2025-06-11 14:12:45,界面新闻,http://stock.eastmoney.com/a/20250611342767276...
4,300059,计算机产业趋势利好积聚，数字经济ETF560800上涨0.53%，成交额超千万元,截至2025年6月9日 10:58，中证数字经济主题指数(931582)上涨0.47%，成分...,2025-06-09 11:14:46,界面新闻,http://stock.eastmoney.com/a/20250609342529155...
...,...,...,...,...,...,...
95,300059,8股特大单净流入资金超2亿元,东方财富 20.69 -0.53 -1.41 非银金融 300065 海兰信 16.52 -...,2025-05-27 18:14:57,证券时报网,http://finance.eastmoney.com/a/202505273415771...
96,300059,资本市场格局有望重塑，金融科技ETF516860深V反弹，新国都涨超12%,数据显示，截至2025年4月30日，中证金融科技主题指数(930986)前十大权重股分别为同...,2025-05-28 11:19:52,界面新闻,http://stock.eastmoney.com/a/20250528341662226...
97,300059,资金流向日报丨东方财富、比亚迪、中航成飞获融资资金买入排名前三,两市共有3398只个股有融资资金买入，其中东方财富、比亚迪、中航成飞排名前三，买入金额分别1...,2025-05-16 11:14:55,21世纪经济报道,http://finance.eastmoney.com/a/202505163406602...
98,300059,成交额超2000万元，金融科技ETF516860近1年涨幅排名可比基金首位,数据显示，截至2025年4月30日，中证金融科技主题指数(930986)前十大权重股分别为同...,2025-05-27 13:36:57,界面新闻,http://stock.eastmoney.com/a/20250527341552651...


| Yahoo Finance | AKShare |
| ------------- | ------- |
| url           | 新闻链接    |
| source        | 文章来源    |
| title         | 新闻标题    |
| date          | 发布时间    |
| text          | 新闻内容    |



In [7]:
for idx, row in stock_news_em_df.iterrows():
    new_content: dict = {}
    print(row["新闻内容"])

截至2025年6月11日 09:47，国证证券龙头指数(399437)成分股信达证券(601059)上涨7.19%，兴业证券(601377)上涨6.76%，中国银河(601881)上涨3.67%，东吴证券(601555)，东方财富300059等个股跟涨。
截至2025年6月11日 09:59，中证800证券保险指数(399966)强势上涨2.72%，成分股信达证券(601059)上涨10.01%，兴业证券(601377)上涨8.78%，东兴证券(601198)上涨5.73%，中国银河(601881)，东方财富300059等个股跟涨
截至2025年6月11日 14:14，中证证券公司30指数(931412)强势上涨2.36%，成分股兴业证券(601377)上涨9.97%，信达证券(601059)上涨9.24%，东兴证券(601198)上涨5.17%，中国银河(601881)，东方财富300059等个股跟涨。
截至2025年6月11日 13:41，中证数字经济主题指数(931582)上涨0.36%，成分股中科曙光(603019)上涨3.22%，同花顺(300033)上涨2.84%，东方财富300059上涨2.38%，均胜电子(600699)上涨1.98%，沪硅产业(688126)上涨
截至2025年6月9日 10:58，中证数字经济主题指数(931582)上涨0.47%，成分股用友网络(600588)上涨6.42%，东方财富300059上涨2.97%，金山办公(688111)上涨2.79%，同花顺(300033)上涨2.55%，润泽科技(300442)上涨2.24%
截至2025年6月4日 10:46，国证证券龙头指数(399437)上涨0.79%，成分股光大证券(601788)上涨5.18%，华泰证券(601688)上涨1.20%，东吴证券(601555)上涨1.00%，中金公司(601995)上涨0.90%，东方财富300059上涨0.82%
截至2025年5月29日 13:26，中证800证券保险指数(399966)强势上涨1.00%，成分股国盛金控(002670)上涨4.25%，湘财股份(600095)上涨2.37%，长城证券(002939)上涨1.65%，东方财富300059，国元证券(000728)等个股跟涨
近日，由中国青年创业就业基金会、东方财富（300